# MCP Registry Demo - Part 2: Deploy ToolHive Operator

This notebook guides you through deploying the Stacklok ToolHive operator on OpenShift.

## What You'll Learn

- How to create a namespace for the MCP registry
- How to deploy the ToolHive operator
- How to verify the operator is running correctly
- How to troubleshoot common deployment issues

## Step 1: Import Libraries and Setup

In [ ]:
import subprocess
import json
import os
import time
from pathlib import Path

def run_command(cmd, show_output=True):
    """Helper function to run shell commands"""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if show_output:
        if result.stdout:
            print(result.stdout)
        if result.stderr and result.returncode != 0:
            print(f"Error: {result.stderr}")
    return result.returncode, result.stdout, result.stderr

# Get repository root
repo_root = Path.cwd().parent
manifest_dir = repo_root / "manifests"
namespace = "mcp-registry"

print(f"Repository root: {repo_root}")
print(f"Manifest directory: {manifest_dir}")
print(f"Target namespace: {namespace}")

## Step 2: Create the Namespace

In [ ]:
print("Creating namespace...\n")

# Apply namespace manifest
namespace_manifest = manifest_dir / "00-namespace.yaml"
returncode, stdout, stderr = run_command(f"oc apply -f {namespace_manifest}")

if returncode == 0:
    print("\n✓ Namespace created successfully")
else:
    print("\n✗ Failed to create namespace")
    print(f"Error: {stderr}")

## Step 3: Verify Namespace Creation

In [ ]:
print("Verifying namespace...\n")

returncode, stdout, stderr = run_command(f"oc get namespace {namespace}")

if returncode == 0:
    print("\n✓ Namespace exists and is active")
else:
    print("\n✗ Namespace not found")

## Step 4: Deploy ToolHive Operator

In [ ]:
print("Deploying ToolHive operator...\n")

# Apply operator manifest
operator_manifest = manifest_dir / "01-toolhive-operator.yaml"
returncode, stdout, stderr = run_command(f"oc apply -f {operator_manifest}")

if returncode == 0:
    print("\n✓ ToolHive operator manifests applied successfully")
else:
    print("\n✗ Failed to deploy ToolHive operator")
    print(f"Error: {stderr}")

## Step 5: Wait for Operator Deployment

In [ ]:
print("Waiting for operator to be ready...\n")

max_attempts = 30
attempt = 0

while attempt < max_attempts:
    returncode, stdout, stderr = run_command(
        f"oc get deployment toolhive-operator -n {namespace} -o json",
        show_output=False
    )
    
    if returncode == 0:
        try:
            deployment = json.loads(stdout)
            status = deployment.get('status', {})
            ready_replicas = status.get('readyReplicas', 0)
            replicas = status.get('replicas', 0)
            
            print(f"Attempt {attempt + 1}/{max_attempts}: {ready_replicas}/{replicas} replicas ready")
            
            if ready_replicas == replicas and replicas > 0:
                print("\n✓ ToolHive operator is ready!")
                break
        except json.JSONDecodeError:
            print(f"Attempt {attempt + 1}/{max_attempts}: Waiting for deployment...")
    
    attempt += 1
    time.sleep(10)
else:
    print("\n⚠ Timeout waiting for operator to be ready")
    print("Check the logs with: oc logs -n mcp-registry deployment/toolhive-operator")

## Step 6: Verify Operator Status

In [ ]:
print("Checking operator deployment status...\n")

# Get deployment details
run_command(f"oc get deployment toolhive-operator -n {namespace}")

print("\nChecking operator pods...\n")

# Get pod details
run_command(f"oc get pods -n {namespace} -l app=toolhive-operator")

## Step 7: View Operator Logs

In [ ]:
print("Fetching operator logs (last 20 lines)...\n")

returncode, stdout, stderr = run_command(
    f"oc logs -n {namespace} deployment/toolhive-operator --tail=20"
)

if returncode != 0:
    print("\n⚠ Could not fetch logs. The operator may not be running yet.")

## Step 8: Verify RBAC Configuration

In [ ]:
print("Checking RBAC configuration...\n")

# Check service account
print("Service Account:")
run_command(f"oc get serviceaccount toolhive-operator -n {namespace}")

print("\nCluster Role:")
run_command("oc get clusterrole toolhive-operator")

print("\nCluster Role Binding:")
run_command("oc get clusterrolebinding toolhive-operator")

## Summary

In this notebook, you:
- Created the `mcp-registry` namespace
- Deployed the ToolHive operator
- Verified the operator is running correctly
- Checked RBAC configurations

### Next Steps

Continue to **03-deploy-mcp-registry.ipynb** to deploy the MCP registry itself.

### Troubleshooting

If the operator is not starting:
1. Check logs: `oc logs -n mcp-registry deployment/toolhive-operator`
2. Check events: `oc get events -n mcp-registry`
3. Verify RBAC: Ensure ClusterRole and ClusterRoleBinding are created
4. Check image pull: Verify the operator image is accessible